In [ ]:
import pandas as pd
import numpy as np
import sys, os
from collections import defaultdict

llpsprots = "../../llps_human_all_proteins.csv"
datafile  = "../../cosmic/CosmicMutantExport.tsv.gz"

mapfile_ENSP = "../llps_uniprot2ENSP.tab.txt"
mapfile_ENST = "../llps_uniprot2ENST.tab.txt"

def load_mapping(mapfile):
    mapdict = defaultdict(lambda: False)
    with open(mapfile) as infmt:
        next(infmt)
        for line in infmt:
            arr = line.strip().split("\t")
            if mapdict[arr[1]]:
                print(f"WARNING: {arr[1]} already in dict")
            else:
                mapdict[arr[1]] = True
    return mapdict
                
ENSP_dict = load_mapping(mapfile_ENSP)
ENST_dict = load_mapping(mapfile_ENST)

In [ ]:
# read COSMIC file line by line, to heavy to load entirely in memory

import gzip

select = ["Gene name", "Accession Number", "HGNC ID", "Primary site", 
          "Primary histology", "Genome-wide screen", "GENOMIC_MUTATION_ID", "LEGACY_MUTATION_ID",
          "MUTATION_ID", "Mutation CDS", "Mutation AA", "Mutation Description", "GRCh", 
          "Mutation genome position", "SNP", "Mutation somatic status", "Pubmed_PMID", "Age",
          "HGVSP", "HGVSC", "HGVSG"]

all_cols = list()
with gzip.open(datafile) as ifile, open("COSMIC_crop.txt", 'w') as ofile:
    headers = next(ifile).decode().rstrip('\n').split("\t")
    ix = [headers.index(x) for x in select]
    ofile.write("\t".join(select)+"\n")
    for i,line in enumerate(ifile):
        try:
            arr = line.decode().rstrip("\n").split("\t")

            cols = [arr[i] for i in ix]
            # print(cols[18], cols[19], cols[20])
            if cols[18] != "p.?" and cols[18] != "":
                ENSP_id = cols[18].split(".")[0]
            if cols[19] != "":
                ENST_id = cols[19].split(".")[0]

            # Filter only mutations in our LLPS dataset
            if ENSP_dict[ENSP_id] or ENST_dict[ENST_id]:
                ofile.write("\t".join(cols)+"\n")        
        except:
            print(f"Error at line {i}: {line}")
            continue

In [1]:
from collections import defaultdict

mut_dict = defaultdict(list)

with open("COSMIC_crop.txt") as ifile:
    headers = next(ifile)
    for line in ifile:
        arr = line.rstrip("\n").split("\t")
        mut_dict[arr[11]].append(arr)
        

In [2]:
for k in mut_dict:
    print(f"{k}: {len(mut_dict[k])} mutations")

Substitution - coding silent: 901732 mutations
Unknown: 10361611 mutations
Substitution - Missense: 3349351 mutations
Substitution - Nonsense: 276396 mutations
Deletion - Frameshift: 140620 mutations
: 6561 mutations
Insertion - Frameshift: 79493 mutations
Deletion - In frame: 48375 mutations
Complex - frameshift: 2332 mutations
Frameshift: 1850 mutations
Insertion - In frame: 13431 mutations
Nonstop extension: 2467 mutations
Complex - compound substitution: 8 mutations
Whole gene deletion: 2088 mutations
Complex - deletion inframe: 364 mutations
Complex - insertion inframe: 52 mutations


In [3]:
import pandas as pd
import re 
import numpy as np

headers = ["Gene name", "Accession Number", "HGNC ID", "Primary site", 
          "Primary histology", "Genome-wide screen", "GENOMIC_MUTATION_ID", "LEGACY_MUTATION_ID",
          "MUTATION_ID", "Mutation CDS", "Mutation AA", "Mutation Description", "GRCh", 
          "Mutation genome position", "SNP", "Mutation somatic status", "Pubmed_PMID", "Age",
          "HGVSP", "HGVSC", "HGVSG"]



In [17]:
def separar_en_cols(df, column, conseq, conseq_regex, override=False):
    '''
    recibe un DataFrame, el nombre de una columna auxiliar (column)
    y un string con el tipo de consecuencia (conseq). La col. auxiliar
    es una tupla con los elementos implicados en una mutacion
    como la siguiente (aa1, start_pos, aa2, end_pos, aa/s_nuevos).
    Devuelve el DataFrame df con estas 5 nuevas columnas
    '''
    
    df_crop = df[df[column].str.contains(conseq_regex)].copy()
      
    if override:
        df_crop['aux'] = df_crop[column].str.findall(conseq_regex).str[0]
    else:
        df_crop['aux'] = df_crop[column].str.findall('^([A-Z][a-z]{2})(\d+)_?([A-Z][a-z]{2})?(\d+)?'+conseq_regex+'(.*)$').str[0]
        
    if conseq == "missense" or conseq == "nonsense":
        df_crop['start_aa'] = df_crop['aux'].map(lambda x: x[1])
        df_crop['end_aa'] = df_crop['start_aa']
        df_crop['from'] = df_crop['aux'].map(lambda x: x[0])        
        df_crop['to'] = df_crop['aux'].map(lambda x: x[2])
    else:
    
        # start position
        df_crop['start_aa'] = df_crop['aux'].map(lambda x: x[1])

        # end position
        df_crop['end_aa'] = df_crop['aux'].map(lambda x: int(x[3]) if x[3] != '' else np.nan)

        # from: es el/los aa que cambian
        df_crop['from'] = df_crop['aux'].map(lambda x: x[0] + x[2]) # concateno si existe mas de un aa que cambia (o sea, si es un rango)

        # to: aa/s nuevos
        if conseq == "nonsense":
            df_crop['to'] = "Ter"
        elif conseq == "deletion":
            df_crop['to'] = ""
        else:
            df_crop['to'] = df_crop['aux'].map(lambda x: x[4] if x[4] != '' else np.nan)

    # consecuencia de la mutacion
    df_crop['consequence'] = conseq

    df_crop = df_crop.drop(columns=['aux'])

    return df_crop[['cambio', 'start_aa', 'end_aa', 'from', 'to', 'consequence']]

def seq3(seq):
    
    protein_letters_1to3 = {
        "A": "Ala",
        "C": "Cys",
        "D": "Asp",
        "E": "Glu",
        "F": "Phe",
        "G": "Gly",
        "H": "His",
        "I": "Ile",
        "K": "Lys",
        "L": "Leu",
        "M": "Met",
        "N": "Asn",
        "P": "Pro",
        "Q": "Gln",
        "R": "Arg",
        "S": "Ser",
        "T": "Thr",
        "V": "Val",
        "W": "Trp",
        "Y": "Tyr",
        "B": "Asx",
        "X": "Xaa",
        "Z": "Glx",
        "J": "Xle",
        "U": "Sec",
        "O": "Pyl",
        "*": "Ter"
    }
    
    return "".join(protein_letters_1to3.get(aa, "Xaa") for aa in seq)

In [23]:

def separar_en_cols_missense(df, column, conseq, conseq_regex, override=False):
    
    df_crop = df[df[column].str.contains(conseq_regex)].copy()
      
    if override:
        df_crop['aux'] = df_crop[column].str.findall(conseq_regex).str[0]
    else:
        df_crop['aux'] = df_crop[column].str.findall('^([A-Z][a-z]{2})(\d+)_?([A-Z][a-z]{2})?(\d+)?'+conseq_regex+'(.*)$').str[0]
            
    # start position
    df_crop['start_aa'] = df_crop['aux'].map(lambda x: x[1])
    df_crop.start_aa = df_crop.start_aa.apply(int)
    
    # end position
    df_crop['end_aa'] = df_crop['start_aa']
    # df_crop['end_aa'] = df_crop['aux'].map(lambda x: int(x[1]) if x[1] != '' else np.nan)
    
    # from: es el/los aa que cambian
    df_crop['from'] = df_crop['aux'].map(lambda x: seq3(x[0])) # concateno si existe mas de un aa que cambia (o sea, si es un rango)
    df_crop['from'] = df_crop['from'].apply(str)
    
    df_crop['to'] = df_crop['aux'].map(lambda x: seq3(x[2]))
    df_crop['to'] = df_crop['to'].apply(str)
        
    # consecuencia de la mutacion
    df_crop['consequence'] = conseq

    df_crop = df_crop.drop(columns=['aux'])

    return df_crop[['cambio', 'start_aa', 'end_aa', 'from', 'to', 'consequence']]

## Process MISSENSE

In [6]:
df_tmp = pd.DataFrame(mut_dict["Substitution - Missense"], columns=headers)

# Subset mutations with "p." only
df_tmp['cambio'] = df_tmp["HGVSP"].map(lambda x: re.findall('p\..*$', x))
df_tmp['cambio'] = df_tmp.cambio.str[0]
df_tmp.cambio = df_tmp.cambio.str.lstrip('p.') 

# separate those that don't have HGVSP
ix_nulls = df_tmp["cambio"].isnull()
df_nulls = df_tmp[ix_nulls].copy()

# do classic missense processing
missense = separar_en_cols(df_tmp[~ix_nulls], "cambio", "missense", '^([A-Z][a-z]{2})(\d+)(?!Ter)([A-Z][a-z]{2})$', override=True)
df_tmp = df_tmp.drop(columns=["cambio"])

# Now process those that have NULL in HGVSP but have some information on "Mutation AA"
# small check in case something is not a missense mutation
check = df_nulls["Mutation AA"].map(lambda x: re.findall('p\.[A-Z]\d+[A-Z]$', x))
if np.sum(check.isnull()) > 0:
    print("Warning MISSENSE! some nulls or errors here")

# continue with mutation processing
ix_X = df_nulls["Mutation AA"].str.contains('p\.[A-Z]\d+X$')  # discard mutations to X
df_pass = df_nulls[~ix_X].copy()
df_pass["cambio"] = df_pass["Mutation AA"].str.lstrip('p.')
df_done = separar_en_cols_missense(df_pass, "cambio", "missense", '^([A-Z])(\d+)([A-Z])$', override=True)

/home/franco/miniconda3/lib/python3.7/site-packages/pandas/core/strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [7]:
all_missense = pd.concat((missense, df_done))
missense_df = pd.merge(df_tmp, all_missense, right_index=True, left_index=True)
print(missense.shape)
print(df_done.shape)
print(all_missense.shape)
print(missense_df.shape)



(3345113, 6)
(1519, 6)
(3346632, 6)
(3346632, 27)


In [11]:
missense_df.to_csv("llps_missense_cosmic.csv.gz", sep=",", header=True, index=False, compression='gzip')

In [12]:
del missense_df
del all_missense

## Process NONSENSE

In [34]:
### Process Nonsense

df_tmp = pd.DataFrame(mut_dict["Substitution - Nonsense"], columns=headers)

# Subset mutations with "p." only
df_tmp['cambio'] = df_tmp["HGVSP"].map(lambda x: re.findall('p\..*$', x))
df_tmp['cambio'] = df_tmp.cambio.str[0]
df_tmp.cambio = df_tmp.cambio.str.lstrip('p.') 

# # separate those that don't have HGVSP
ix_nulls = df_tmp["cambio"].isnull()
df_nulls = df_tmp[ix_nulls].copy()

# # do classic missense processing
nonsense = separar_en_cols(df_tmp[~ix_nulls], "cambio", "nonsense", "(?<=\d)Ter", override=False)
df_tmp = df_tmp.drop(columns=["cambio"])

# # Now process those that have NULL in HGVSP but have some information on "Mutation AA"
# # small check in case something is not a missense mutation
check = df_nulls["Mutation AA"].map(lambda x: re.findall('p\.[A-Z]\d+\*$', x))
if np.sum(check.isnull()) > 0:
    print("Warning NONSENSE! some nulls or errors here")

# # continue with mutation processing
df_nulls["cambio"] = df_nulls["Mutation AA"].str.lstrip('p.')
df_done = separar_en_cols_missense(df_nulls, "cambio", "nonsense", '^([A-Z])(\d+)(\*)$', override=True)

/home/franco/miniconda3/lib/python3.7/site-packages/pandas/core/strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [35]:
print(nonsense.shape)
print(df_done.shape)

(275729, 6)
(212, 6)


In [36]:
all_nonsense = pd.concat((nonsense, df_done))
nonsense_df = pd.merge(df_tmp, all_nonsense, right_index=True, left_index=True)

In [37]:
nonsense_df.to_csv("llps_nonsense_cosmic.csv.gz", sep=",", header=True, index=False, compression='gzip')

In [38]:
print(nonsense_df.shape)

(275941, 27)


In [39]:
del nonsense_df
del all_nonsense

# Process DELETIONS

In [166]:
### Process Deletions

df_tmp = pd.DataFrame(mut_dict["Deletion - In frame"], columns=headers)
df_tmp

# Subset mutations with "p." only
df_tmp['cambio'] = df_tmp["HGVSP"].map(lambda x: re.findall('p\..*$', x))
df_tmp['cambio'] = df_tmp.cambio.str[0]
df_tmp.cambio = df_tmp.cambio.str.lstrip('p.') 

# separate those that don't have HGVSP
ix_nulls = df_tmp["cambio"].isnull()
df_nulls = df_tmp[ix_nulls]
df_notnulls = df_tmp[~ix_nulls].copy()
ix_right = df_notnulls["cambio"].str.contains('^([A-Z][a-z]{2})(\d+)_?([A-Z][a-z]{2})?(\d+)?del$')
df_weird = df_notnulls[~ix_right].copy()

# # # do classic missense processing
deletions = separar_en_cols(df_notnulls[ix_right], "cambio", "deletion", '^([A-Z][a-z]{2})(\d+)_?([A-Z][a-z]{2})?(\d+)?del$', override=True)
# df_tmp = df_tmp.drop(columns=["cambio"])

# # # Now process those that have NULL in HGVSP but have some information on "Mutation AA"
# # # small check in case something is not a missense mutation
# check = df_nulls["Mutation AA"].map(lambda x: re.findall('p\.[A-Z]\d+\*$', x))
# if np.sum(check.isnull()) > 0:
#     print("Warning NONSENSE! some nulls or errors here")

# # # continue with mutation processing
# df_nulls["cambio"] = df_nulls["Mutation AA"].str.lstrip('p.')
# df_done = separar_en_cols_missense(df_nulls, "cambio", "nonsense", '^([A-Z])(\d+)(\*)$', override=True)

/home/franco/miniconda3/lib/python3.7/site-packages/pandas/core/strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [169]:
deletions[:50]

,cambio,start_aa,end_aa,from,to,consequence
0,Lys359del,359,NaN,Lys,,deletion
1,Lys359del,359,NaN,Lys,,deletion
2,Lys359del,359,NaN,Lys,,deletion
3,Gln394del,394,NaN,Gln,,deletion
4,Lys359del,359,NaN,Lys,,deletion
5,Lys359del,359,NaN,Lys,,deletion
7,Glu208del,208,NaN,Glu,,deletion
8,Lys359del,359,NaN,Lys,,deletion
9,Glu208del,208,NaN,Glu,,deletion
10,Ala2del,2,NaN,Ala,,deletion


In [165]:
def separar_en_cols(df, column, conseq, conseq_regex, override=False):
    '''
    recibe un DataFrame, el nombre de una columna auxiliar (column)
    y un string con el tipo de consecuencia (conseq). La col. auxiliar
    es una tupla con los elementos implicados en una mutacion
    como la siguiente (aa1, start_pos, aa2, end_pos, aa/s_nuevos).
    Devuelve el DataFrame df con estas 5 nuevas columnas
    '''
    
    df_crop = df[df[column].str.contains(conseq_regex)].copy()
      
    if override:
        df_crop['aux'] = df_crop[column].str.findall(conseq_regex).str[0]
    else:
        df_crop['aux'] = df_crop[column].str.findall('^([A-Z][a-z]{2})(\d+)_?([A-Z][a-z]{2})?(\d+)?'+conseq_regex+'(.*)$').str[0]
        
    if conseq == "missense" or conseq == "nonsense":
        df_crop['start_aa'] = df_crop['aux'].map(lambda x: x[1])
        df_crop['end_aa'] = df_crop['start_aa']
        df_crop['from'] = df_crop['aux'].map(lambda x: x[0])        
        df_crop['to'] = df_crop['aux'].map(lambda x: x[2])
    else:
    
        # start position
        df_crop['start_aa'] = df_crop['aux'].map(lambda x: x[1])

        # end position
        df_crop['end_aa'] = df_crop['aux'].map(lambda x: int(x[3]) if x[3] != '' else np.nan)

        # from: es el/los aa que cambian
        df_crop['from'] = df_crop['aux'].map(lambda x: x[0] + x[2]) # concateno si existe mas de un aa que cambia (o sea, si es un rango)

        # to: aa/s nuevos
        if conseq == "nonsense":
            df_crop['to'] = "Ter"
        elif conseq == "deletion":
            df_crop['to'] = ""
        else:
            df_crop['to'] = df_crop['aux'].map(lambda x: x[4] if x[4] != '' else np.nan)

    # consecuencia de la mutacion
    df_crop['consequence'] = conseq

    df_crop = df_crop.drop(columns=['aux'])

    return df_crop[['cambio', 'start_aa', 'end_aa', 'from', 'to', 'consequence']]


In [151]:
conseq_regex = 'del'
for e in df_notnulls[ix_right]["cambio"]:
    m = re.search('^([A-Z][a-z]{2})(\d+)_?([A-Z][a-z]{2})?(\d+)?del$', e)
    if m is not None:
        print(m.group(1), m.group(2), m.group(3), m.group(4))
    else:
        print(m)

Lys 359 None None
Lys 359 None None
Lys 359 None None
Gln 394 None None
Lys 359 None None
Lys 359 None None
Glu 208 None None
Lys 359 None None
Glu 208 None None
Ala 2 None None
Gln 394 None None
Glu 208 None None
Met 357 Lys 388
Lys 150 None None
Tyr 346 None None
Met 357 Lys 388
Met 357 Lys 388
Tyr 346 None None
Val 267 Asn 270
Lys 359 None None
Ile 1200 None None
Arg 331 None None
Thr 421 Pro 422
Met 357 Lys 388
Gly 35 Glu 38
Gly 35 Glu 38
His 354 Gln 357
Gln 161 None None
Cys 460 Ala 462
Glu 414 None None
Ser 337 Lys 340
Gln 161 None None
Tyr 346 None None
Leu 141 None None
Gln 161 None None
Leu 165 None None
Gln 161 None None
Gln 161 None None
Gln 161 None None
Lys 336 None None
Arg 331 None None
Pro 280 Ala 284
Glu 1923 None None
Glu 1923 None None
Glu 1923 None None
Glu 967 None None
Ile 1200 None None
Glu 967 None None
Glu 1388 None None
Glu 1923 None None
Glu 1620 None None
Thr 245 None None
Val 371 None None
Glu 1923 None None
Lys 1622 None None
Glu 1923 None None
Glu 1388 No

Gln 204 None None
Ile 240 None None
Ile 240 None None
Gln 204 None None
Gln 204 None None
Gln 204 None None
Gln 204 None None
Gln 203 Gln 204
Gln 204 None None
Gln 204 None None
Gln 204 None None
Gln 204 None None
Leu 116 Leu 122
Gln 204 None None
Gln 204 None None
Gln 204 None None
Gln 204 None None
Gln 204 None None
Gln 204 None None
Gln 204 None None
His 375 Pro 382
Gln 204 None None
Pro 88 None None
Gln 204 None None
Gln 204 None None
Gln 204 None None
Phe 1467 None None
Gln 204 None None
Gln 204 None None
Gln 204 None None
Gln 204 None None
Ile 820 None None
Gln 204 None None
Gln 204 None None
Gln 204 None None
Gln 204 None None
Gln 204 None None
His 375 Pro 382
Gln 204 None None
Lys 658 None None
Trp 896 None None
His 1703 Arg 1708
Glu 1301 None None
His 1703 Arg 1708
Lys 129 None None
Lys 129 None None
His 1703 Arg 1708
Glu 1509 None None
Gly 313 None None
His 523 None None
Gly 71 None None
Ser 68 None None
Arg 110 None None
Gly 327 None None
Lys 1595 None None
Phe 1467 None Non

Lys 297 None None
Gln 345 None None
Glu 104 None None
Leu 234 Gly 241
Asp 466 None None
Lys 101 None None
Val 730 None None
Glu 1464 None None
Asn 1768 Glu 1775
Gly 149 None None
Glu 1464 None None
Lys 297 None None
Asp 1264 None None
Thr 49 None None
Asn 60 Arg 63
Thr 49 None None
Gln 2194 None None
Glu 1414 None None
Glu 1464 None None
Lys 297 None None
Glu 631 None None
Thr 1529 None None
Ser 1206 None None
Ser 69 None None
Glu 64 None None
Glu 64 None None
Glu 598 None None
Pro 948 Thr 989
Glu 1464 None None
Glu 1464 None None
Leu 670 None None
Glu 1464 None None
Lys 297 None None
Pro 948 Thr 989
Glu 25 None None
Thr 1529 None None
Ser 141 None None
Lys 297 None None
Lys 455 None None
Arg 185 None None
Glu 1464 None None
Glu 1464 None None
Lys 297 None None
Lys 297 None None
Gln 309 None None
Ser 1206 None None
Lys 2225 None None
Glu 1464 None None
Tyr 313 None None
Asp 1264 None None
Arg 141 None None
Arg 141 None None
Ser 1110 None None
Ser 764 Gln 769
Ala 128 Glu 130
Leu 456 Non

Gln 507 Gln 510
Gln 649 Gln 650
Ser 45 None None
Leu 31 Ser 33
Leu 46 Ser 47
Gln 650 None None
Gln 772 None None
Thr 40 Ala 43
Gln 772 None None
Gln 647 Gln 650
Gln 507 Gln 510
Ala 5 Ala 80
Gln 507 Gln 510
Gln 507 Gln 510
Arg 178 Leu 180
Ala 5 Ala 80
Ser 45 None None
Gln 507 Gln 510
Gln 508 Gln 510
Gln 650 None None
Arg 281 Gly 283
Gly 38 Pro 44
Gln 507 Gln 510
Leu 724 None None
Gln 507 Gln 510
Gln 772 None None
Gln 507 Gln 510
Gln 507 Gln 510
Gln 772 None None
Gln 772 None None
Gln 509 Gln 510
Trp 25 Ser 33
Gln 772 None None
Gln 507 Gln 510
Ser 23 Ala 39
Ala 5 Ala 80
Ser 45 None None
Gln 507 Gln 510
Gln 650 None None
Ile 35 None None
Ser 45 None None
Gln 507 Gln 510
Ser 881 His 889
Ala 5 Ala 80
Leu 828 None None
Gln 772 None None
Asp 6 Ser 29
Gln 772 None None
Ser 45 None None
Gln 772 None None
Gln 507 Gln 510
Ala 5 Ala 80
Ala 5 Ala 80
Ala 5 Ala 80
Ser 45 None None
Ala 5 Ala 80
Ala 5 Ala 80
His 36 Ser 37
Ala 5 Ala 80
Gln 507 Gln 510
Gln 507 Gln 510
Gln 772 None None
Ser 23 Ser 33
Ser 

Gly 490 Gly 493
Gly 24 None None
Gly 490 Gly 493
Glu 1331 None None
Phe 28 None None
Gly 490 Gly 493
Gly 24 None None
Gly 490 Gly 493
Ser 700 None None
Gly 24 None None
Ser 508 Gly 512
Tyr 542 Gly 543
Glu 454 None None
Ser 553 None None
Lys 588 None None
Gly 490 Gly 493
Gly 243 Pro 244
Gly 490 Gly 493
Gly 490 Gly 493
Ser 508 Gly 512
Gly 490 Gly 493
Gly 24 None None
Ser 483 None None
Gln 371 None None
Gly 490 Gly 493
Lys 689 None None
Thr 55 Gly 61
Lys 16 Ala 17
Glu 464 None None
His 1287 None None
Glu 666 None None
His 1287 None None
Arg 875 None None
Phe 1234 Lys 1237
Lys 689 None None
Glu 639 None None
Val 558 None None
Glu 1212 None None
Glu 1331 None None
Val 558 None None
Lys 517 None None
Lys 1434 None None
Lys 689 None None
Leu 1101 None None
Lys 517 None None
Gly 243 Pro 244
Gln 424 None None
Thr 2407 None None
Gln 424 None None
Lys 344 None None
Val 1275 None None
Glu 1268 Glu 1273
Glu 1273 None None
Glu 1273 None None
Glu 1268 Glu 1273
Gln 424 None None
Gln 424 None None
Glu 

...

Ala 5 Ala 80
Ser 45 None None
Leu 275 None None
Leu 46 Ser 47
Lys 186 None None
Ser 45 None None
Ala 5 Ala 80
Val 157 Ile 162
Glu 294 None None
Trp 25 Asp 32
Cys 176 His 179
Cys 44 His 47
Ser 274 Ser 290
Met 307 None None
Glu 22 None None
Ser 45 None None
Glu 50 Pro 51
Leu 708 Val 718
Lys 1329 None None
Gly 302 Gly 303
Lys 1048 None None
Ala 6 Val 11
Ala 138 Val 143
Glu 372 None None
Met 105 Asn 107
Met 237 Asn 239
Ile 123 None None
Ile 255 None None
Arg 52 Pro 59
Pro 16942 None None
Leu 98 None None
Gly 271 Gly 275
Tyr 30 Ser 33
Thr 155 Arg 156
Phe 79 Phe 113
Met 90 Pro 91
Asp 104 Ile 106
Glu 719 None None
Leu 31 Ser 33
Ser 45 None None
Ser 45 None None
Pro 58 His 115
Leu 31 Ile 35
His 469 None None
Glu 286 None None
Pro 45 Cys 50
Ser 32604 None None
Glu 388 None None
Gln 1238 None None
Lys 689 None None
Ser 45 Ser 47
Ser 45 None None
Pro 177 Cys 182
Lys 269 None None
Ser 45 None None
Gln 95 None None
Ser 45 None None
Ala 4558 Ala 4563
Asp 148 Thr 150
Asp 16 Thr 18
Asp 32 None None
Se

Pro 45 Cys 50
Glu 17518 None None
Leu 666 None None
Lys 110 None None
Ala 145 Tyr 147
Thr 772 Ala 773
His 619 None None
Lys 377 None None
Lys 206 None None
Ser 940 None None
Ser 7 Gln 11
Glu 424 None None
Phe 25 Ser 26
Pro 1261 None None
Leu 67 None None
Ser 261 None None
Val 25 None None
Glu 377 None None
Gly 427 Gly 429
Ala 5 Ala 80
Gly 262 None None
Ile 255 None None
Leu 265 None None
Gly 4 Gly 17
Asn 131 None None
Asn 107 Ser 108
Ala 5 Ala 80
Gln 95 None None
Pro 485 None None
Gly 262 None None
Gly 6 None None
Ala 5 Ala 80
Ala 5 Ala 80
Glu 388 None None
Met 90 Pro 91
Leu 133 None None
Cys 176 None None
Pro 177 Cys 182
Leu 1209 Gln 1216
Ala 5 Ala 80
Ser 129 None None
Val 108 None None
Glu 666 None None
Asp 149 Arg 150
Asp 149 Arg 150
Arg 429 None None
Leu 324 Pro 326
Ser 45 None None
Pro 177 Cys 182
Ser 45 None None
Gly 262 None None
Gly 108 Phe 109
Ala 5 Ala 80
Val 86 None None
Pro 59 None None
Glu 180 Asp 184
Gln 1216 None None
Gln 1216 None None
Glu 50 None None
Glu 168 None None

Phe 242 None None
Gly 240 None None
Val 218 None None
Ile 123 None None
Thr 124 None None
Ser 106 Tyr 107
Glu 275 None None
Ala 5 Ala 80
Glu 153 Asn 156
Leu 327 Leu 328
Val 40 Glu 48
Tyr 104 None None
Tyr 104 None None
Asn 567 None None
Met 90 Pro 91
Glu 15644 Ile 15647
Asn 255 Val 256
Tyr 236 Met 243
Gly 199 Glu 204
Pro 177 Cys 182
Pro 177 Cys 182
Val 378 Phe 379
Tyr 280 None None
Gln 12 None None
Ser 128 Ser 129
Ser 45 None None
Leu 265 None None
Glu 210 None None
Pro 21 Gly 22
Ile 122 Thr 124
Ser 1096 Glu 1098
Val 173 His 179
Asn 263 Phe 270
Gln 33 None None
Leu 252 None None
Gln 373 None None
Glu 266 None None
Glu 945 None None
Glu 462 None None
Leu 265 None None
Ala 49 Thr 53
Thr 41 Ser 45
Arg 117 Pro 118
Ser 45 None None
Thr 422 Tyr 423
Leu 470 None None
Asn 103 Tyr 104
His 650 None None
Gln 1238 None None
Lys 28689 Glu 28690
Gly 480 Pro 482
Glu 153 Asn 156
Leu 13 None None
Leu 65 None None
Leu 252 Ile 254
Val 218 None None
Pro 158 Met 161
Asn 131 None None
Val 157 Arg 158
Gly 10

Ser 157 None None
Ile 255 None None
Gln 192 None None
Met 237 Asn 239
Leu 705 Thr 709
Thr 155 Arg 156
Thr 155 Arg 156
Gly 648 None None
Lys 1023 None None
Lys 909 None None
Arg 203 Leu 213
Glu 325 None None
Phe 453 None None
His 179 Glu 180
Gln 144 None None
Ile 255 None None
Ser 1023 None None
Thr 150 Arg 158
Gly 470 None None
Lys 1358 None None
Glu 1257 None None
Pro 142 Gln 144
Arg 151 None None
Ser 29 None None
Glu 330 None None
Met 105 Asn 107
Tyr 236 None None
Tyr 236 None None
Gly 130 None None
Gly 130 None None
Gln 12 None None
Lys 462 Asp 465
Gly 130 None None
Gly 16 Gly 17
Thr 211 Ser 215
Asn 107 Ser 108
Cys 97 Ile 100
Phe 453 None None
Ser 1270 None None
Gly 108 Phe 109
Ile 255 None None
Thr 1401 None None
Lys 588 None None
Glu 462 None None
Asp 63 Gln 108
Glu 40 None None
Glu 287 None None
Ser 215 Tyr 220
Pro 59 Leu 69
Ala 8 None None
Glu 1805 None None
Tyr 126 Lys 132
Pro 45 Cys 50
Lys 453 None None
Leu 252 Ile 254
Glu 25468 None None
Asp 1431 None None
Gln 524 None None
V

Lys 546 None None
Lys 562 None None
Gln 144 None None
Tyr 102 None None
Ser 109 None None
Gly 147 Arg 148
Ile 2940 None None
Lys 1409 None None
Arg 249 Thr 256
Gln 524 None None
Arg 26 Ala 27
Val 20681 Thr 20682
Pro 191 None None
Cys 141 Pro 142
Glu 286 Glu 287
Thr 150 Pro 151
Gly 68 None None
Gly 416 Val 421
Gly 266 Asn 268
Ser 307 None None
His 164 Ser 171
Leu 145 None None
Glu 210 None None
Gln 373 None None
Ile 123 None None
Ile 123 None None
Glu 204 Asn 210
Arg 7 None None
Glu 210 None None
Pro 191 None None
Val 218 None None
Lys 1227 None None
Cys 106 None None
Glu 154 Glu 155
Pro 177 His 178
Glu 204 Leu 206
Val 218 None None
Arg 592 None None
Gly 2760 None None
Phe 109 Arg 110
Pro 448 None None
Val 218 None None
Thr 150 None None
Val 218 None None
Lys 361 Asp 364
Pro 191 None None
Pro 59 None None
Gly 62 None None
Gln 373 None None
Glu 429 None None
Leu 5 Gln 10
Gln 373 None None
Gln 373 None None
Gln 373 None None
Ile 3093 None None
Tyr 31 Gln 33
Pro 45 Cys 50
Val 86 None None


In [108]:
df_tmp[~ix_nulls][:14]

,Gene name,Accession Number,HGNC ID,Primary site,Primary histology,Genome-wide screen,GENOMIC_MUTATION_ID,LEGACY_MUTATION_ID,MUTATION_ID,Mutation CDS,...,GRCh,Mutation genome position,SNP,Mutation somatic status,Pubmed_PMID,Age,HGVSP,HGVSC,HGVSG,cambio
0,GATA3,ENST00000379328.8,4172,breast,carcinoma,n,COSV60516010,COSM6904322,114423927,c.1075_1077del,...,38,10:8073763-8073765,n,Confirmed somatic variant,28481359,,ENSP00000368632.3:p.Lys359del,ENST00000379328.8:c.1075_1077del,10:g.8073763_8073765del,Lys359del
1,GATA3,ENST00000379328.8,4172,breast,carcinoma,n,COSV60516010,COSM6904322,114423927,c.1075_1077del,...,38,10:8073763-8073765,n,Reported in another cancer sample as somatic,30062102,,ENSP00000368632.3:p.Lys359del,ENST00000379328.8:c.1075_1077del,10:g.8073763_8073765del,Lys359del
2,GATA3,ENST00000379328.8,4172,breast,carcinoma,n,COSV60516010,COSM6904322,114423927,c.1075_1077del,...,38,10:8073763-8073765,n,Confirmed somatic variant,30205045,66,ENSP00000368632.3:p.Lys359del,ENST00000379328.8:c.1075_1077del,10:g.8073763_8073765del,Lys359del
3,TP73_ENST00000604479,ENST00000604479.5,12003,large_intestine,carcinoma,y,COSV60698986,COSM4735502,159429521,c.1181_1183del,...,38,1:3729433-3729435,n,Confirmed somatic variant,25344691,64.16,ENSP00000474322.1:p.Gln394del,ENST00000604479.5:c.1181_1183del,1:g.3729433_3729435del,Gln394del
4,GATA3,ENST00000379328.8,4172,breast,carcinoma,n,COSV60516010,COSM6904322,114423927,c.1075_1077del,...,38,10:8073763-8073765,n,Reported in another cancer sample as somatic,30181556,,ENSP00000368632.3:p.Lys359del,ENST00000379328.8:c.1075_1077del,10:g.8073763_8073765del,Lys359del
5,GATA3,ENST00000379328.8,4172,breast,carcinoma,n,COSV60516010,COSM6904322,114423927,c.1075_1077del,...,38,10:8073763-8073765,n,Reported in another cancer sample as somatic,30181556,,ENSP00000368632.3:p.Lys359del,ENST00000379328.8:c.1075_1077del,10:g.8073763_8073765del,Lys359del
6,CKAP2_ENST00000490903,ENST00000490903.5,1990,large_intestine,carcinoma,y,COSV51621004,COSM5111120,135489931,c.653_661del,...,38,13:52461623-52461631,n,Confirmed somatic variant,22810696,81,ENSP00000417830.1:p.Thr218_Gln221delinsLys,ENST00000490903.5:c.653_661del,13:g.52461623_52461631del,Thr218_Gln221delinsLys
7,CHMP3,ENST00000263856.8,29865,NS,malignant_melanoma,y,COSV55686973,COSM5862948,98776512,c.618_620del,...,38,2:86505864-86505866,n,Confirmed somatic variant,24265153,47,ENSP00000263856.4:p.Glu208del,ENST00000263856.8:c.618_620del,2:g.86505864_86505866del,Glu208del
8,GATA3,ENST00000379328.8,4172,breast,carcinoma,n,COSV60516010,COSM6904322,114423927,c.1075_1077del,...,38,10:8073763-8073765,n,Confirmed somatic variant,30205045,64,ENSP00000368632.3:p.Lys359del,ENST00000379328.8:c.1075_1077del,10:g.8073763_8073765del,Lys359del
9,CHMP3,ENST00000263856.8,29865,NS,malignant_melanoma,y,COSV55686973,COSM5862948,98776512,c.618_620del,...,38,2:86505864-86505866,n,Confirmed somatic variant,24265153,57,ENSP00000263856.4:p.Glu208del,ENST00000263856.8:c.618_620del,2:g.86505864_86505866del,Glu208del


In [95]:
df_tmp[~ix_nulls]["cambio"][:40]

0                         Lys359del
1                         Lys359del
2                         Lys359del
3                         Gln394del
4                         Lys359del
5                         Lys359del
6            Thr218_Gln221delinsLys
7                         Glu208del
8                         Lys359del
9                         Glu208del
10                          Ala2del
11                        Gln394del
12                        Glu208del
13                 Met357_Lys388del
14                        Lys150del
15                        Tyr346del
16                 Met357_Lys388del
17                 Met357_Lys388del
18                        Tyr346del
19                 Val267_Asn270del
20                        Lys359del
21                       Ile1200del
22                        Arg331del
23                 Thr421_Pro422del
24                 Met357_Lys388del
25                   Gly35_Glu38del
26                   Gly35_Glu38del
27                 His354_Gl

In [163]:
# list(df_tmp[~ix_nulls]["cambio"].str.findall('^([A-Z][a-z]{2})(\d+)_?([A-Z][a-z]{2})?(\d+)?'+conseq_regex+'(.*)$').str[0][0:100])
aux = df_notnulls[ix_right]["cambio"].str.findall('^([A-Z][a-z]{2})(\d+)_?([A-Z][a-z]{2})?(\d+)?del$').str[0][0:100]
for x in aux:
    print(x[4])

IndexError: tuple index out of range